In [1]:
import tensorflow as tf

2025-10-04 16:23:10.242232: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-04 16:23:10.525948: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-04 16:23:11.984115: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
class AlexNet(tf.keras.Model):
    def __init__ (self, num_classes=10):
        super().__init__()
        self.net = tf.keras.Sequential([
            tf.keras.layers.Conv2D(filters=96, kernel_size=11, strides=4, activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=256, kernel_size=5, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(filters=384, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding='same'),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dense(4096, activation='relu'),
            tf.keras.layers.Dense(num_classes)
        ])
    def call(self, x):
        return self.net(x)
        

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.expand_dims(image, axis=-1)
    image = tf.image.resize(image, [224, 224])
    return image, label

batch_size = 128

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.map(preprocess) \
                   .shuffle(buffer_size=1024) \
                   .batch(batch_size) \
                   .prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.map(lambda x, y: (tf.cast(x, tf.float32)/255.0, y)) \
                 .batch(batch_size) \
                 .prefetch(tf.data.AUTOTUNE)

In [8]:
model = AlexNet()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
              metrics=['accuracy'])

In [10]:
model.fit(train_ds, epochs=10)

Epoch 1/10


2025-10-04 16:37:04.700726: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fe500016150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-04 16:37:04.700742: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-10-04 16:37:04.723845: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-04 16:37:04.832575: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301
2025-10-04 16:37:05.521060: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_135', 8 bytes spill stores, 8 bytes spill loads

2025-10-04 16:37:06.461049: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:

  1/469 ━━━━━━━━━━━━━━━━━━━━ 2:21:37 18s/step - accuracy: 0.0625 - loss: 2.3011

I0000 00:00:1759588641.823524   10857 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.4408 - loss: 1.6573

2025-10-04 16:38:07.805519: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_135', 8 bytes spill stores, 8 bytes spill loads

2025-10-04 16:38:08.026489: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_121', 12 bytes spill stores, 12 bytes spill loads

2025-10-04 16:38:08.565191: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_135', 8 bytes spill stores, 8 bytes spill loads

2025-10-04 16:38:08.614334: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_121', 32 bytes spill stores, 32 bytes spill loads

2025-10-04 16:38:08.743373: I external/local_xla

469/469 ━━━━━━━━━━━━━━━━━━━━ 78s 129ms/step - accuracy: 0.5994 - loss: 1.1420
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.7925 - loss: 0.5546
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 99ms/step - accuracy: 0.8325 - loss: 0.4510
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 99ms/step - accuracy: 0.8539 - loss: 0.3973
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 99ms/step - accuracy: 0.8668 - loss: 0.3620
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 99ms/step - accuracy: 0.8766 - loss: 0.3338
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 99ms/step - accuracy: 0.8848 - loss: 0.3146
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.8923 - loss: 0.2968
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.8966 - loss: 0.2809
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.9015 - loss: 0.2669
